In [1]:
import json
import requests
token = "xxx"
headers = {'Authorization': "Token " + token}
rate_limit_url = "https://api.github.com/rate_limit"
req = requests.get(rate_limit_url, headers=headers).json()
print(req)

{'resources': {'core': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1664813490}, 'search': {'limit': 30, 'used': 0, 'remaining': 30, 'reset': 1664809950}, 'graphql': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1664813490}, 'integration_manifest': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1664813490}, 'source_import': {'limit': 100, 'used': 0, 'remaining': 100, 'reset': 1664809950}, 'code_scanning_upload': {'limit': 1000, 'used': 0, 'remaining': 1000, 'reset': 1664813490}, 'actions_runner_registration': {'limit': 10000, 'used': 0, 'remaining': 10000, 'reset': 1664813490}, 'scim': {'limit': 15000, 'used': 0, 'remaining': 15000, 'reset': 1664813490}, 'dependency_snapshots': {'limit': 100, 'used': 0, 'remaining': 100, 'reset': 1664809950}}, 'rate': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1664813490}}


In [1]:
# import json
# import requests
# import sys
# import re

# change_ids = ['Iff7ee24159d54b578cd854e620aa814d72da3801', 'Idbabd162fa9e0dbce687981bdbcc75be37189a61', 'I99ff7863a509262ad9d4f7c9c5afbc66fd37001c', 'I086077934aee6dc940705f08a87bf8448708881f', 'I4fed19a1d3bb2247656b052306307e6980074405', 'Ic7b27f78678eeaa87a38607af787f254db1383b8', 'I9e9f7861257ad5bce46a0cf113d1a9d7824911e6', 'I01e74a01c87b381ee7cd6f20d51a1fa61c0e98fc', 'I1aa11a5fbbe66ede4ec2e5b2090044a39052a241', 'Ic31017b3f2671108adb6d6118ef1c75f1ccc3ec5', 'Iacb6a2c8d5e880b16b0c8f0c9257ed94bed36f5b', 'Idd95d1a3aa057d23e3adb6635dd1acbb2c853497', 'I0faeb5f9b1a226aada958c18333d9c2ac8203dd1', 'I76294a9156173314a3cf09160d0ca4e0d7c6ef3a', 'Ie577bda046f60ce23dfb8cdd6164bb3de4b54358']
# # change_ids = ['I086077934aee6dc940705f08a87bf8448708881f']
# bot_users = [1000049, 1003699, 1007413]

# def get_change_latest_revision(id, owner):
#   latest_revision = 1
#   nmw = False # Need More Work
#   message_url = "https://codereview.qt-project.org/changes/qt%2Fqtbase~dev~{}/messages".format(id)
#   try:
#     res = requests.get(message_url)
#     if res.status_code == 200:
#       messages = json.loads(res.text[5:-1])
#       for msg_item in messages:
#         tag = msg_item["tag"] if "tag" in msg_item else ""
#         author = msg_item["author"]["_account_id"]
#         message = msg_item["message"]
#         revision_number = msg_item["_revision_number"]
#         autogenerated = re.search("^autogenerated:.*", tag)
#         comment = re.search(".*\(\d* comments?\).*", message)
#         review_minus_1 = re.search(".*Code-Review-1.*", message)
#         review_minus_2 = re.search(".*Code-Review-2.*", message)
#         if author != owner and author not in bot_users:
#           if comment or review_minus_1 or review_minus_2:
#             nmw = True
#             break
#         if not nmw:
#           latest_revision = max(latest_revision, revision_number)
#     else:
#       print("Unexpected response code from REST server for change {}: {}".format(id, res.status_code), file=sys.stderr)
#   except Exception as e:
#     print("Unexpected exception for change {}: {}".format(id, e), file=sys.stderr)
#   return latest_revision, nmw


# for id in change_ids:
#   url = "https://codereview.qt-project.org/changes/qt%2Fqtbase~dev~{}".format(id)
#   try:
#     res = requests.get(url)
#     if res.status_code == 200:
#       data = json.loads(res.text[5:-1])
#       owner = data["owner"]["_account_id"]
#       latest_revision, nmw = get_change_latest_revision(id, owner)
#       print("id: {}, latest revision: {}, nmw: {}".format(id, latest_revision, nmw))
#     else:
#       print("Unexpected response code from REST server for change {}: {}".format(id, res.status_code), file=sys.stderr)
#   except Exception as e:
#     print("Unexpected exception for change {}: {}".format(id, e), file=sys.stderr)

In [2]:
import json
import requests
import sys
import pandas as pd

changes_df = pd.read_csv('apache_commons_lang_experiment_changes.csv')
# display(changes_df)
files_df = pd.read_csv('apache_commons_lang_experiment_files.csv')

users = {}
changes = []
files = []
authors = []

def get_change_info(id):
  change = {}
  url = "https://api.github.com/repos/apache/commons-lang/commits/{}".format(id)
  try:
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
      data = res.json()
    #   data = json.loads(res.text[6:-2])
      change = {
        "change_id": data["sha"],
        "repo": "apache/commons-lang",
        "branch": "master",
        "subject": None,
        "created": data["commit"]["author"]["date"],
        "updated": data["commit"]["committer"]["date"],
        "submitted": None,
        "insertions": data["stats"]["additions"],
        "deletions": data["stats"]["deletions"],
        "number": None,
        "author": data["commit"]["author"]["email"],
        "commit_msg": data["commit"]["message"],
        "parent": data["parents"][0]["sha"],
        "project": "apache"
      }
      if data["commit"]["author"]["email"] not in users:
          users[data["commit"]["author"]["email"]] = {
            "name": data["commit"]["author"]["name"],
            "email": data["commit"]["author"]["email"],
          }
    else:
      print("Unexpected response code from REST server for change {}: {}".format(id, res.status_code), file=sys.stderr)
  except Exception as e:
    print("Unexpected exception for change {}: {}".format(id, e), file=sys.stderr)
  return change

def get_changed_files(id):
  files = []
  url = "https://api.github.com/repos/apache/commons-lang/commits/{}".format(id)
  try:
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
      data = res.json()
      parent = data["parents"][0]["sha"]
      for file in data["files"]:
        raw_url = file["raw_url"].replace(id, parent)
        file_content = requests.get(raw_url, headers=headers).text
        files.append({
          "filename": file["filename"],
          "status": None,
          "insertions": file["additions"],
          "deletions": file["deletions"],
          "code_a": file_content,
          "code_b": None,
          "diff": "--- a/{}\n +++ b/{}\n".format(file["filename"], file["filename"]) + file["patch"]
        })
    else:
      print("Unexpected response code from REST server for change {}: {}".format(id, res.status_code), file=sys.stderr)
  except Exception as e:
    print("Unexpected exception for change {}: {}".format(id, e), file=sys.stderr)
  return files

for index, row in files_df.iterrows():
  file = {
    "id": index + 1,
    "change_id": row['commit_id'],
    "filename": row['file'],
    "prior_bugs": row['file_prior_bugs'],
    "prior_changes": row['file_prior_changes']
  }
  change_files = get_changed_files(row['commit_id'])
  for f in change_files:
    if f["filename"] == row['file']:
      file.update(f)
  files.append(file)

for index, row in changes_df.iterrows():
  id = row['commit_id']
  change = get_change_info(id)
  change["author_prior_changes"] = row["author_prior_changes"]
  change["author_prior_bugs"] = row["author_prior_bugs"]
  change["risk_score"] = row["risk_density"]
  change["bug_density"] = row["bug_density"]
  change['practice'] = row["practice"]
  changes.append(change)

author_id = 1
users_new = {}
for email, info in users.items():
  users_new[email] = {
    "account_id": author_id,
    "name": info["name"],
    "email": info["email"],
    "project": "apache"
  }
  author_id += 1
users = users_new
authors = list(users.values())

print(changes)
# print(files)
print(users)
print(len(changes))
print(len(files))
print(len(authors))
# print(file_diff['src/corelib/thread/qsemaphore.cpp']['code_a'])



[{'change_id': '188933345a7ebad94f74ba0fb6e8bc6eb99552a6', 'repo': 'apache/commons-lang', 'branch': 'master', 'subject': None, 'created': '2013-10-11T11:25:28Z', 'updated': '2013-10-11T11:25:28Z', 'submitted': None, 'insertions': 2, 'deletions': 2, 'number': None, 'author': 'britter@apache.org', 'commit_msg': 'Reuse functionality already present and fix PMD violations an the way\n\ngit-svn-id: https://svn.apache.org/repos/asf/commons/proper/lang/trunk@1531257 13f79535-47bb-0310-9956-ffa450edef68', 'parent': '960608021348739df69ec64ebd7fbe2455fed269', 'project': 'apache', 'author_prior_changes': 17, 'author_prior_bugs': 0, 'risk_score': 0.823148815136705, 'bug_density': 0.0, 'practice': 1}, {'change_id': '65392be352be6ccc8acf24405d819f60cd0d1a22', 'repo': 'apache/commons-lang', 'branch': 'master', 'subject': None, 'created': '2014-02-06T13:48:08Z', 'updated': '2014-02-06T13:48:08Z', 'submitted': None, 'insertions': 2, 'deletions': 0, 'number': None, 'author': 'sebb@apache.org', 'commit_

In [ ]:
# import mysql.connector
# from itertools import cycle

# cnx = mysql.connector.connect(user='root', password='password',
#                               host='localhost',
#                               database='gherald')


# cursor = cnx.cursor()
# add_author = ("INSERT INTO author "
#                "(account_id, email, name, username, project) "
#                "VALUES (%s, %s, %s, %s, %s)")
# add_change = ("INSERT INTO change_detail "
#                "(id, branch, commit_msg, created, deletions, insertions, number, parent, repo, subject, submitted, updated, author_account_id, project, author_prior_changes, author_prior_bugs, risk_score, bug_density) "
#                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
# add_file = ("INSERT INTO file "
#                "(id, codea, codeb, diff, deletions, filename, insertions, status, change_id, prior_changes, prior_bugs) "
#                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")

# for author in authors:
#   data_author = (author["account_id"], author["email"], author["name"], "None", author["project"])
#   cursor.execute(add_author, data_author)

# for change in changes:
#   data_change = (change["change_id"], change["branch"], change["commit_msg"], change["created"], change["deletions"], change["insertions"], change["number"], change["parent"], change["repo"], change["subject"], change["submitted"], change["updated"], change["author"], change["project"], change["author_prior_changes"], change["author_prior_bugs"], change["risk_score"], change["bug_density"])
#   cursor.execute(add_change, data_change)

# for file in files:
#   data_file = (file["id"], file["code_a"], file["code_b"], file["diff"], file["deletions"], file["filename"], file["insertions"], file["status"], file["change_id"], file["prior_changes"], file["prior_bugs"])
#   cursor.execute(add_file, data_file)

# # val = [("Nikhil", "CSE", "98", "A", "18"),
# #        ("Nisha", "CSE", "99", "A", "18"),
# #        ("Rohan", "MAE", "43", "B", "20"),
# #        ("Amit", "ECE", "24", "A", "21"),
# #        ("Anil", "MAE", "45", "B", "20"),
# #        ("Megha", "ECE", "55", "A", "22"),
# #        ("Sita", "CSE", "95", "A", "19")]
   
# # cursorObject.executemany(sql, val)

# cnx.commit()

# cursor.close()
# cnx.close()

In [3]:
import mysql.connector
from itertools import cycle

cnx = mysql.connector.connect(user='root', password='password',
                              host='localhost',
                              database='gherald')
# cnx = mysql.connector.connect(user='b72b48e88d6984', password='7d05d18b',
#                               host='us-cdbr-east-06.cleardb.net',
#                               database='heroku_50897a0d4de5500')


cursor = cnx.cursor()
add_author = ("INSERT INTO author "
               "(account_id, email, name, username, project) "
               "VALUES (%s, %s, %s, %s, %s)")
add_change = ("INSERT INTO change_detail "
               "(id, branch, commit_msg, created, deletions, insertions, number, parent, repo, subject, submitted, updated, author_account_id, project, author_prior_changes, author_prior_bugs, risk_score, bug_density, practice) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
add_file = ("INSERT INTO file "
               "(id, codea, codeb, diff, deletions, filename, insertions, status, change_id, prior_changes, prior_bugs) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")

for author in authors:
  data_author = (author["account_id"], author["email"], author["name"], "None", author["project"])
  cursor.execute(add_author, data_author)

for change in changes:
  data_change = (change["change_id"], change["branch"], change["commit_msg"], change["created"], change["deletions"], change["insertions"], change["number"], change["parent"], change["repo"], change["subject"], change["submitted"], change["updated"], users[change["author"]]["account_id"], change["project"], change["author_prior_changes"], change["author_prior_bugs"], change["risk_score"], change["bug_density"], change["practice"])
  cursor.execute(add_change, data_change)

for file in files:
  data_file = (file["id"], file["code_a"], file["code_b"], file["diff"], file["deletions"], file["filename"], file["insertions"], file["status"], file["change_id"], file["prior_changes"], file["prior_bugs"])
  cursor.execute(add_file, data_file)

# val = [("Nikhil", "CSE", "98", "A", "18"),
#        ("Nisha", "CSE", "99", "A", "18"),
#        ("Rohan", "MAE", "43", "B", "20"),
#        ("Amit", "ECE", "24", "A", "21"),
#        ("Anil", "MAE", "45", "B", "20"),
#        ("Megha", "ECE", "55", "A", "22"),
#        ("Sita", "CSE", "95", "A", "19")]
   
# cursorObject.executemany(sql, val)

cnx.commit()

cursor.close()
cnx.close()

In [4]:
import pandas as pd
import numpy as np

methods_df = pd.read_csv('apache_commons_lang_experiment_methods.csv')
# display(methods_df)
lines_df = pd.read_csv('apache_commons_lang_experiment_lines.csv')

# methods_df = methods_df[(methods_df['commit_id'] == "0de9354f505f415c7ad35fe735cebab4900a698a") | (methods_df['commit_id'] == "bb95f8a19470f38587780d5b29387d328b9160f6") | (methods_df['commit_id'] == "b78295b4faca2fd26024a26ef4638b0dce5d40a3")]
# lines_df = lines_df[(lines_df['commit_id'] == "0de9354f505f415c7ad35fe735cebab4900a698a") | (lines_df['commit_id'] == "bb95f8a19470f38587780d5b29387d328b9160f6") | (lines_df['commit_id'] == "b78295b4faca2fd26024a26ef4638b0dce5d40a3")]


def get_file_id(row):
    filename = row['file']
    commit_id = row['commit_id']
    index = files_df.index[(files_df['file'] == filename) & (files_df['commit_id'] == commit_id)].tolist()[0] + 1
    return index

methods_df['file_id'] = methods_df.apply(lambda x: get_file_id(x), axis=1)
# methods_df = methods_df[['name', 'start_line', 'end_line', 'prior_changes', 'prior_bugs', 'file_id']]
methods_df = methods_df[['method', 'method_start_line', 'method_end_line', 'method_prior_changes', 'method_prior_bugs', 'file_id', 'commit_id']]
methods_df.columns = ['name', 'start_line', 'end_line', 'prior_changes', 'prior_bugs', 'file_id', 'change_id']
lines_df['file_id'] = lines_df.apply(lambda x: get_file_id(x), axis=1)
lines_df = lines_df[['line_number', 'code', 'risk_score', 'file_id', 'commit_id']]
lines_df.columns = ['line_number', 'code', 'risk_score', 'file_id', 'change_id']
methods_df.index = np.arange(1, len(methods_df)+1)
# methods_df.index.name = 'id'
lines_df.index = np.arange(1, len(lines_df)+1)
# lines_df.index.name = 'id'

display(methods_df)
display(lines_df)



,name,start_line,end_line,prior_changes,prior_bugs,file_id,change_id
1,"StringUtils::rightPad( String str , int size ,...",4586,4615,13,0,1,16774d1c0d6d8aa4579f7c96b3fdb78bd118e5aa
2,"StringUtils::leftPad( String str , int size , ...",4698,4727,13,0,1,16774d1c0d6d8aa4579f7c96b3fdb78bd118e5aa
3,StringUtils::normalizeSpace( String str),6478,6483,4,0,1,16774d1c0d6d8aa4579f7c96b3fdb78bd118e5aa
4,"StringUtils::center( String str , int size , S...",4837,4852,11,0,1,16774d1c0d6d8aa4579f7c96b3fdb78bd118e5aa
5,NumberUtils::createBigInteger( String str),695,719,6,0,2,c1f9320476ab9e5f262fdf8a5b3e1ff70199aed8
6,NumberUtils::createNumber( String str),445,604,17,2,3,d844d1eb5e5b530a82b77302f1f284fd2f924be3
7,"CharSetUtils::containsAny( final String str , ...",106,117,0,0,4,bb95f8a19470f38587780d5b29387d328b9160f6
8,CharUtils::isAsciiAlphanumeric(final char ch),535,537,1,0,5,188933345a7ebad94f74ba0fb6e8bc6eb99552a6
9,CharUtils::isAsciiAlpha(final char ch),459,461,1,0,5,188933345a7ebad94f74ba0fb6e8bc6eb99552a6
10,BooleanUtils::isOneTrue( final Boolean ... array),205,218,0,0,6,0de9354f505f415c7ad35fe735cebab4900a698a


,line_number,code,risk_score,file_id,change_id
1,9222,if ( isEmpty ( str ) || isEmpty ( wrapToken ) ...,4,11,06aea7e74cfe4a1578cb76672f1562132090c205
2,9180,if ( isEmpty ( str ) || wrapChar == CharUtils....,2,11,06aea7e74cfe4a1578cb76672f1562132090c205
3,2597,"appendDetail ( buffer , fieldName , try_value_...",20,10,cac520971206093d32bf17925f880bd06d93f736
4,2590,|| try_value_str.startsWith ( getArrayStart ( ...,6,10,cac520971206093d32bf17925f880bd06d93f736
5,2589,if ( try_value_str.startsWith ( getContentStar...,3,10,cac520971206093d32bf17925f880bd06d93f736
...,...,...,...,...,...
146,4842,padStr = SPACE ;,0,1,16774d1c0d6d8aa4579f7c96b3fdb78bd118e5aa
147,460,return isAsciiAlphaUpper ( ch ) || isAsciiAlph...,0,5,188933345a7ebad94f74ba0fb6e8bc6eb99552a6
148,536,return isAsciiAlpha ( ch ) || isAsciiNumeric (...,0,5,188933345a7ebad94f74ba0fb6e8bc6eb99552a6
149,630,default :,0,7,65392be352be6ccc8acf24405d819f60cd0d1a22


In [45]:
display(lines_df[:50])

,line_number,code,risk_score,file_id,change_id
1,9222,if ( isEmpty ( str ) || isEmpty ( wrapToken ) ...,4,11,06aea7e74cfe4a1578cb76672f1562132090c205
2,9180,if ( isEmpty ( str ) || wrapChar == CharUtils....,2,11,06aea7e74cfe4a1578cb76672f1562132090c205
3,2597,"appendDetail ( buffer , fieldName , try_value_...",20,10,cac520971206093d32bf17925f880bd06d93f736
4,2590,|| try_value_str.startsWith ( getArrayStart ( ...,6,10,cac520971206093d32bf17925f880bd06d93f736
5,2589,if ( try_value_str.startsWith ( getContentStar...,3,10,cac520971206093d32bf17925f880bd06d93f736
6,2578,"appendValueAsString ( buffer , value.toString ...",2,10,cac520971206093d32bf17925f880bd06d93f736
7,2588,final String try_value_str = value.toString ( ) ;,2,10,cac520971206093d32bf17925f880bd06d93f736
8,2576,if ( value instanceof String || value instance...,0,10,cac520971206093d32bf17925f880bd06d93f736
9,2579,return ;,0,10,cac520971206093d32bf17925f880bd06d93f736
10,2580,},0,10,cac520971206093d32bf17925f880bd06d93f736


In [29]:
display(files_df)

,commit_id,author_email,author_date,additions,deletions,bug_count,author_prior_changes,author_prior_bugs,file,file_complexity,file_prior_changes,file_prior_bugs
0,16774d1c0d6d8aa4579f7c96b3fdb78bd118e5aa,ggregory@apache.org,2012-11-08 16:08:23,5,4,0.0,541,24,src/main/java/org/apache/commons/lang3/StringU...,726.0,351,11
1,c1f9320476ab9e5f262fdf8a5b3e1ff70199aed8,sebb@apache.org,2012-11-13 00:59:22,21,2,1.0,354,13,src/main/java/org/apache/commons/lang3/math/Nu...,250.0,61,4
2,d844d1eb5e5b530a82b77302f1f284fd2f924be3,sebb@apache.org,2012-11-13 01:39:40,14,6,1.0,355,15,src/main/java/org/apache/commons/lang3/math/Nu...,249.0,62,6
3,bb95f8a19470f38587780d5b29387d328b9160f6,bayard@apache.org,2013-04-20 09:05:43,12,0,0.0,1231,53,src/main/java/org/apache/commons/lang3/CharSet...,32.0,59,0
4,188933345a7ebad94f74ba0fb6e8bc6eb99552a6,britter@apache.org,2013-10-11 11:25:28,2,2,0.0,17,0,src/main/java/org/apache/commons/lang3/CharUti...,46.0,33,2
5,0de9354f505f415c7ad35fe735cebab4900a698a,britter@apache.org,2013-10-15 19:09:31,33,0,0.0,28,0,src/main/java/org/apache/commons/lang3/Boolean...,176.0,62,1
6,65392be352be6ccc8acf24405d819f60cd0d1a22,sebb@apache.org,2014-02-06 13:48:08,2,0,0.0,413,17,src/main/java/org/apache/commons/lang3/Boolean...,165.0,67,1
7,9397608dd35a335d5e14813c0923f9419782980a,sebb@apache.org,2014-02-11 02:53:03,25,18,1.0,417,17,src/main/java/org/apache/commons/lang3/math/Nu...,240.0,76,8
8,b78295b4faca2fd26024a26ef4638b0dce5d40a3,britter@apache.org,2014-05-07 18:42:33,28,0,0.0,246,3,src/main/java/org/apache/commons/lang3/StringU...,840.0,402,14
9,cac520971206093d32bf17925f880bd06d93f736,792626757@qq.com,2015-05-01 06:13:13,14,3,1.0,3,0,src/main/java/org/apache/commons/lang3/builder...,284.0,77,4


In [5]:
from sqlalchemy import create_engine


engine = create_engine("mysql+pymysql://{user}:{password}@{host}/{database}"
				.format(host='localhost', database='gherald', user='root', password='password'))
# engine = create_engine("mysql+pymysql://{user}:{password}@{host}/{database}"
# 				.format(host='us-cdbr-east-06.cleardb.net', database='heroku_50897a0d4de5500', user='b72b48e88d6984', password='7d05d18b'))

methods_df.to_sql('method', engine, if_exists='replace', index_label='id')
lines_df.to_sql('line', engine, if_exists='replace', index_label='id')


In [ ]:
# from datetime import datetime, timezone
# from pydriller import Repository
# import re
# import csv
# import pandas as pd

# files = []
# for commit in Repository('~/Files/Repos/apache/commons-lang', only_in_branch='master', order='date-order').traverse_commits():
#     # files = []
#     # for file in commit.modified_files:
#     #   files.append(file.new_path)
#     if commit.hash == "25e91c4242283d118ba978d4f38cd18e10c890dc":
#         for file in commit.modified_files:
#             filename = file.new_path
#             if file.change_type.name == 'DELETE':
#                 filename = file.old_path
#             methods = []
#             for m in file.changed_methods:
#                 methods.append((m.name, m.long_name, m.start_line, m.end_line))
#             files.append({
#                 'filename': filename,
#                 'methods': methods,
#             })

# print(files)

# # c18803a47f4a4e51f08c1cb1ef9b84b21e106e6f  16ed9eb220802405bef8c49ae937e9256a82b586  cf7c9522aa99d0d180e54ac5abe849f19d153d25

In [2]:
data = {
              '06aea7e74cfe4a1578cb76672f1562132090c205': 
              {
                'author_prior_changes': 1,
                'author_prior_bugs': 0,
                'risk_score': 0.449,
                'bug_density': 0.25,
                'loc': 8,
                'la': 3,
                'diff': 4,
                'bugs': 1,
                'file_prior_bugs': 22,
                'file_prior_changes': 358,
                'method_prior_bugs': 0.5,
                'method_prior_changes': 5,
                'method_complexity': 7,
                'risky_lines': 3,
                'risk_score': 7,
                'files': {
                  'src/main/java/org/apache/commons/lang3/StringUtils.java': {
                    'id': 1,
                    'prior_changes': 358,
                    'prior_bugs': 47
                  },
                  'src/test/java/org/apache/commons/lang3/StringUtilsTest.java': {
                    'id': 2,
                    'prior_changes': 146,
                    'prior_bugs': 0
                  }
                }
              }, 
              'cac520971206093d32bf17925f880bd06d93f736':
              {
                'author_prior_changes': 3,
                'author_prior_bugs': 0,
                'risk_score': 0.1238,
                'bug_density': 0.083,
                'loc': 17,
                'la': 14,
                'bugs': 1,
                'diff': 12,
                'file_prior_bugs': 4,
                'file_prior_changes': 20,
                'method_prior_bugs': 1,
                'method_prior_changes': 2,
                'method_complexity': 10,
                'risky_lines': 4,
                'risk_score': 20,
                'files': {
                  'src/main/java/org/apache/commons/lang3/builder/ToStringStyle.java': {
                    'id': 3,
                    'prior_changes': 20,
                    'prior_bugs': 7
                  }
                }
              }, 
              '9397608dd35a335d5e14813c0923f9419782980a':
              {
                'author_prior_changes': 417,
                'author_prior_bugs': 40,
                'risk_score': 0.1467,
                'bug_density': 0.111,
                'loc': 43,
                'la': 25,
                'bugs': 1,
                'diff': 9,
                'file_prior_bugs': 12,
                'file_prior_changes': 34,
                'method_prior_bugs': 2,
                'method_prior_changes': 3,
                'method_complexity': 52,
                'risky_lines': 2,
                'risk_score': 4,
                'files': {
                  'src/main/java/org/apache/commons/lang3/math/NumberUtils.java': {
                    'id': 4,
                    'prior_changes': 34,
                    'prior_bugs': 13
                  }, 
                  'src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java': {
                    'id': 5,
                    'prior_changes': 37,
                    'prior_bugs': 0
                  }, 
                  'src/changes/changes.xml': {
                    'id': 6,
                    'prior_changes': 110,
                    'prior_bugs': 0
                  }
                }
              }, 
              '0942ef04f1fa911d382b79a92dca1dd4b3eb88e6':
              {
                'author_prior_changes': 131,
                'author_prior_bugs': 3,
                'risk_score': 0.7313,
                'bug_density': 1,
                'loc': 4,
                'la': 2,
                'bugs': 2,
                'diff': 2,
                'file_prior_bugs': 6,
                'file_prior_changes': 31,
                'method_prior_bugs': 1,
                'method_prior_changes': 2,
                'method_complexity': 12,
                'risky_lines': 1,
                'risk_score': 3,
                'files': {
                  'src/main/java/org/apache/commons/lang3/time/DateUtils.java': {
                    'id': 7,
                    'prior_changes': 31,
                    'prior_bugs': 11
                  }, 
                  'src/changes/changes.xml': {
                    'id': 8,
                    'prior_changes': 90,
                    'prior_bugs': 0
                  }, 
                  'src/test/java/org/apache/commons/lang3/time/DateUtilsFragmentTest.java': {
                    'id': 9,
                    'prior_changes': 3,
                    'prior_bugs': 0
                  }
                }
              }, 
              'd844d1eb5e5b530a82b77302f1f284fd2f924be3':
              {
                'author_prior_changes': 355,
                'author_prior_bugs': 35,
                'risk_score': 0.0483,
                'bug_density': 0.143,
                'loc': 20,
                'la': 14,
                'bugs': 2,
                'diff': 14,
                'file_prior_bugs': 8,
                'file_prior_changes': 20,
                'method_prior_bugs': 2,
                'method_prior_changes': 17,
                'method_complexity': 55,
                'risky_lines': 3,
                'risk_score': 4,
                'files': {
                  'src/main/java/org/apache/commons/lang3/math/NumberUtils.java': {
                    'id': 10,
                    'prior_changes': 20,
                    'prior_bugs': 8
                  }, 
                  'src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java': {
                    'id': 11,
                    'prior_changes': 21,
                    'prior_bugs': 0
                  }, 
                  'src/changes/changes.xml': {
                    'id': 12,
                    'prior_changes': 28,
                    'prior_bugs': 0
                  }
                }
              }, 
              'c1f9320476ab9e5f262fdf8a5b3e1ff70199aed8':
              {
                'author_prior_changes': 354,
                'author_prior_bugs': 33,
                'risk_score': 0.0277,
                'bug_density': 0.053,
                'loc': 20,
                'la': 19,
                'bugs': 1,
                'diff': 19,
                'file_prior_bugs': 6,
                'file_prior_changes': 19,
                'method_prior_bugs': 0,
                'method_prior_changes': 6,
                'method_complexity': 9,
                'risky_lines': 0,
                'risk_score': 0,
                'files': {
                  'src/main/java/org/apache/commons/lang3/math/NumberUtils.java': {
                    'id': 13,
                    'prior_changes': 19,
                    'prior_bugs': 7
                  }, 
                  'src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java': {
                    'id': 14,
                    'prior_changes': 20,
                    'prior_bugs': 0
                  }, 
                  'src/changes/changes.xml': {
                    'id': 15,
                    'prior_changes': 27,
                    'prior_bugs': 0
                  }
                }
              },
              '9379d0d36acdf78455e81518b3b1476c7691f056':
              {
                'author_prior_changes': 15,
                'author_prior_bugs': 0,
                'risk_score': 0.053,
                'bug_density': 0,
                'loc': 12,
                'la': 12,
                'bugs': 0,
                'diff': 12,
                'file_prior_bugs': 10,
                'file_prior_changes': 109,
                'method_prior_bugs': 0,
                'method_prior_changes': 0,
                'method_complexity': 2.5,
                'risky_lines': 0,
                'risk_score': 0,
                'files': {
                  'src/main/java/org/apache/commons/lang3/ArrayUtils.java': {
                    'id': 16,
                    'prior_changes': 109,
                    'prior_bugs': 10
                  }, 
                  'src/test/java/org/apache/commons/lang3/ArrayUtilsTest.java': {
                    'id': 17,
                    'prior_changes': 62,
                    'prior_bugs': 0
                  }
                }
              },
              '0de9354f505f415c7ad35fe735cebab4900a698a':
              {
                'author_prior_changes': 28,
                'author_prior_bugs': 0,
                'risk_score': 0.0221,
                'bug_density': 0,
                'loc': 33,
                'la': 33,
                'bugs': 0,
                'diff': 33,
                'file_prior_bugs': 1,
                'file_prior_changes': 26,
                'method_prior_bugs': 0,
                'method_prior_changes': 0,
                'method_complexity': 5.5,
                'risky_lines': 3,
                'risk_score': 5,
                'files': {
                  'src/main/java/org/apache/commons/lang3/BooleanUtils.java': {
                    'id': 18,
                    'prior_changes': 26,
                    'prior_bugs': 2
                  }, 
                  'src/test/java/org/apache/commons/lang3/BooleanUtilsTest.java': {
                    'id': 19,
                    'prior_changes': 14,
                    'prior_bugs': 0
                  }
                }
              },
              'bb95f8a19470f38587780d5b29387d328b9160f6':
              {
                'author_prior_changes': 1231,
                'author_prior_bugs': 81,
                'risk_score': 0.0687,
                'bug_density': 0,
                'loc': 12,
                'la': 12,
                'bugs': 0,
                'diff': 12,
                'file_prior_bugs': 0,
                'file_prior_changes': 12,
                'method_prior_bugs': 0,
                'method_prior_changes': 0,
                'method_complexity': 5,
                'risky_lines': 3,
                'risk_score': 4,
                'files': {
                  'src/changes/changes.xml': {
                    'id': 20,
                    'prior_changes': 40,
                    'prior_bugs': 0
                  }, 
                  'src/main/java/org/apache/commons/lang3/CharSetUtils.java': {
                    'id': 21,
                    'prior_changes': 12,
                    'prior_bugs': 0
                  },
                  'src/test/java/org/apache/commons/lang3/CharSetUtilsTest.java': {
                    'id': 22,
                    'prior_changes': 5,
                    'prior_bugs': 0
                  }
                }
              },
              '4f85c164a1a4eeb8813b61cf46132fb91971b323':
              {
                'author_prior_changes': 461,
                'author_prior_bugs': 5,
                'risk_score': 0.0509,
                'bug_density': 0,
                'loc': 10,
                'la': 10,
                'bugs': 0,
                'diff': 10,
                'file_prior_bugs': 23,
                'file_prior_changes': 418,
                'method_prior_bugs': 0,
                'method_prior_changes': 0,
                'method_complexity': 3,
                'risky_lines': 0,
                'risk_score': 0,
                'files': {
                  'src/changes/changes.xml': {
                    'id': 23,
                    'prior_changes': 742,
                    'prior_bugs': 0
                  }, 
                  'src/main/java/org/apache/commons/lang3/StringUtils.java': {
                    'id': 24,
                    'prior_changes': 418,
                    'prior_bugs': 49
                  },
                  'src/test/java/org/apache/commons/lang3/StringUtilsSubstringTest.java': {
                    'id': 25,
                    'prior_changes': 16,
                    'prior_bugs': 0
                  }
                }
              },
              'b78295b4faca2fd26024a26ef4638b0dce5d40a3':
              {
                'author_prior_changes': 246,
                'author_prior_bugs': 6,
                'risk_score': 0.0224,
                'bug_density': 0,
                'loc': 29,
                'la': 29,
                'bugs': 0,
                'diff': 29,
                'file_prior_bugs': 18,
                'file_prior_changes': 189,
                'method_prior_bugs': 0,
                'method_prior_changes': 0,
                'method_complexity': 9,
                'risky_lines': 0,
                'risk_score': 0,
                'files': {
                  'src/changes/changes.xml': {
                    'id': 26,
                    'prior_changes': 146,
                    'prior_bugs': 0
                  }, 
                  'src/main/java/org/apache/commons/lang3/StringUtils.java': {
                    'id': 27,
                    'prior_changes': 189,
                    'prior_bugs': 36
                  },
                  'src/test/java/org/apache/commons/lang3/StringUtilsSubstringTest.java': {
                    'id': 28,
                    'prior_changes': 72,
                    'prior_bugs': 0
                  }
                }
              }}


data_list = data.values()
risk_scores = []
bug_density = []

for data in data_list:
  author_score = (1231-data['author_prior_changes'])/1231 * ((data['author_prior_bugs']+1) / (data['author_prior_changes']+1))
  # author_score =  ((data['author_prior_bugs']+1) / (data['author_prior_changes']+1))
  # file_score = data['file_prior_bugs'] / data['file_prior_changes']
  file_score = (418-data['file_prior_changes'])/418 * ((data['file_prior_bugs']+1) / (data['file_prior_changes']+1))
  # method_score = 0 if data['method_prior_changes'] == 0 else data['method_prior_bugs'] / data['method_prior_changes']
  method_score = data['method_complexity']/55 if data['method_prior_changes'] == 0 else (17-data['method_prior_changes'])/17 * ((data['method_prior_bugs']+1) / (data['method_prior_changes']+1))
  line_score = (data['risky_lines'] / data['la'])
  score = author_score + file_score + method_score + line_score
  # score = author_score + file_score + line_score
  risk_scores.append(score / data['diff'])
  bug_density.append(data['bugs'] / data['diff'])

print(risk_scores)
print(bug_density)

[0.4213151533243356, 0.11250362004235744, 0.12263592069368039, 0.6589202320767742, 0.04959375423205912, 0.026040026989374437, 0.015093034539676123, 0.008911336655045093, 0.03463532081953135, 0.006266893139354553, 0.00831371454512896]
[0.25, 0.08333333333333333, 0.1111111111111111, 1.0, 0.14285714285714285, 0.05263157894736842, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# import json
# import requests
# import sys

# # change_ids = ['I9bd5f80ada856b7db4b39dfb59b32bd825416c13']
# # change_ids = ['I32172ed44d74378c627918e19b9e1aaadb5c6d1d', 'I9b352ac98e2a961157f5bb36456bec3e35891270', 'I419c324634be8ee6884e02032bb53a42738305ac']
# data = {
#               '06aea7e74cfe4a1578cb76672f1562132090c205': 
#               {
#                 'author_prior_changes': 1,
#                 'author_prior_bugs': 0,
#                 'risk_score': 0.449,
#                 'bug_density': 0.25,
#                 'loc': 8,
#                 'la': 3,
#                 'diff': 4,
#                 'bugs': 1,
#                 'file_prior_bugs': 47,
#                 'file_prior_changes': 358,
#                 'method_prior_bugs': 2,
#                 'method_prior_changes': 6,
#                 'risky_lines': 3,
#                 'files': {
#                   'src/main/java/org/apache/commons/lang3/StringUtils.java': {
#                     'id': 1,
#                     'prior_changes': 358,
#                     'prior_bugs': 47
#                   },
#                   'src/test/java/org/apache/commons/lang3/StringUtilsTest.java': {
#                     'id': 2,
#                     'prior_changes': 146,
#                     'prior_bugs': 0
#                   }
#                 }
#               }, 
#               'cac520971206093d32bf17925f880bd06d93f736':
#               {
#                 'author_prior_changes': 3,
#                 'author_prior_bugs': 0,
#                 'risk_score': 0.1238,
#                 'bug_density': 0.083,
#                 'loc': 17,
#                 'la': 14,
#                 'bugs': 1,
#                 'diff': 12,
#                 'file_prior_bugs': 7,
#                 'file_prior_changes': 20,
#                 'method_prior_bugs': 1,
#                 'method_prior_changes': 2,
#                 'risky_lines': 4,
#                 'files': {
#                   'src/main/java/org/apache/commons/lang3/builder/ToStringStyle.java': {
#                     'id': 3,
#                     'prior_changes': 20,
#                     'prior_bugs': 7
#                   }
#                 }
#               }, 
#               '9397608dd35a335d5e14813c0923f9419782980a':
#               {
#                 'author_prior_changes': 417,
#                 'author_prior_bugs': 30,
#                 'risk_score': 0.1467,
#                 'bug_density': 0.111,
#                 'loc': 43,
#                 'la': 25,
#                 'bugs': 1,
#                 'diff': 9,
#                 'file_prior_bugs': 13,
#                 'file_prior_changes': 34,
#                 'method_prior_bugs': 3,
#                 'method_prior_changes': 3,
#                 'risky_lines': 2,
#                 'files': {
#                   'src/changes/changes.xml': {
#                     'id': 4,
#                     'prior_changes': 110,
#                     'prior_bugs': 0
#                   },
#                   'src/main/java/org/apache/commons/lang3/math/NumberUtils.java': {
#                     'id': 5,
#                     'prior_changes': 34,
#                     'prior_bugs': 13
#                   }, 
#                   'src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java': {
#                     'id': 6,
#                     'prior_changes': 37,
#                     'prior_bugs': 0
#                   }
#                 }
#               },
#               '0942ef04f1fa911d382b79a92dca1dd4b3eb88e6':
#               {
#                 'author_prior_changes': 131,
#                 'author_prior_bugs': 3,
#                 'risk_score': 0.7313,
#                 'bug_density': 1,
#                 'loc': 4,
#                 'la': 2,
#                 'bugs': 2,
#                 'diff': 2,
#                 'file_prior_bugs': 11,
#                 'file_prior_changes': 31,
#                 'method_prior_bugs': 1,
#                 'method_prior_changes': 2,
#                 'risky_lines': 1,
#                 'files': {
#                   'src/changes/changes.xml': {
#                     'id': 7,
#                     'prior_changes': 90,
#                     'prior_bugs': 0
#                   }, 
#                   'src/main/java/org/apache/commons/lang3/time/DateUtils.java': {
#                     'id': 8,
#                     'prior_changes': 31,
#                     'prior_bugs': 11
#                   }, 
#                   'src/test/java/org/apache/commons/lang3/time/DateUtilsFragmentTest.java': {
#                     'id': 9,
#                     'prior_changes': 3,
#                     'prior_bugs': 0
#                   }
#                 }
#               }, 
#               'd844d1eb5e5b530a82b77302f1f284fd2f924be3':
#               {
#                 'author_prior_changes': 355,
#                 'author_prior_bugs': 26,
#                 'risk_score': 0.0483,
#                 'bug_density': 0.143,
#                 'loc': 20,
#                 'la': 14,
#                 'bugs': 2,
#                 'diff': 14,
#                 'file_prior_bugs': 8,
#                 'file_prior_changes': 20,
#                 'method_prior_bugs': 2,
#                 'method_prior_changes': 17,
#                 'risky_lines': 3,
#                 'files': {
#                   'src/changes/changes.xml': {
#                     'id': 10,
#                     'prior_changes': 28,
#                     'prior_bugs': 0
#                   },
#                   'src/main/java/org/apache/commons/lang3/math/NumberUtils.java': {
#                     'id': 11,
#                     'prior_changes': 20,
#                     'prior_bugs': 8
#                   }, 
#                   'src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java': {
#                     'id': 12,
#                     'prior_changes': 21,
#                     'prior_bugs': 0
#                   }
#                 }
#               }, 
#               'c1f9320476ab9e5f262fdf8a5b3e1ff70199aed8':
#               {
#                 'author_prior_changes': 354,
#                 'author_prior_bugs': 25,
#                 'risk_score': 0.0277,
#                 'bug_density': 0.053,
#                 'loc': 20,
#                 'la': 19,
#                 'bugs': 1,
#                 'diff': 19,
#                 'file_prior_bugs': 7,
#                 'file_prior_changes': 19,
#                 'method_prior_bugs': 0,
#                 'method_prior_changes': 6,
#                 'risky_lines': 0,
#                 'files': {
#                   'src/changes/changes.xml': {
#                     'id': 13,
#                     'prior_changes': 27,
#                     'prior_bugs': 0
#                   },
#                   'src/main/java/org/apache/commons/lang3/math/NumberUtils.java': {
#                     'id': 14,
#                     'prior_changes': 19,
#                     'prior_bugs': 7
#                   }, 
#                   'src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java': {
#                     'id': 15,
#                     'prior_changes': 20,
#                     'prior_bugs': 0
#                   }
#                 }
#               },
#               '0de9354f505f415c7ad35fe735cebab4900a698a':
#               {
#                 'author_prior_changes': 28,
#                 'author_prior_bugs': 0,
#                 'risk_score': 0.0221,
#                 'bug_density': 0,
#                 'loc': 33,
#                 'la': 33,
#                 'bugs': 0,
#                 'diff': 33,
#                 'file_prior_bugs': 2,
#                 'file_prior_changes': 26,
#                 'method_prior_bugs': 0,
#                 'method_prior_changes': 0,
#                 'risky_lines': 3,
#                 'files': {
#                   'src/main/java/org/apache/commons/lang3/BooleanUtils.java': {
#                     'id': 18,
#                     'prior_changes': 26,
#                     'prior_bugs': 2
#                   }, 
#                   'src/test/java/org/apache/commons/lang3/BooleanUtilsTest.java': {
#                     'id': 19,
#                     'prior_changes': 14,
#                     'prior_bugs': 0
#                   }
#                 }
#               },
#               'bb95f8a19470f38587780d5b29387d328b9160f6':
#               {
#                 'author_prior_changes': 1231,
#                 'author_prior_bugs': 88,
#                 'risk_score': 0.0687,
#                 'bug_density': 0,
#                 'loc': 12,
#                 'la': 12,
#                 'bugs': 0,
#                 'diff': 12,
#                 'file_prior_bugs': 0,
#                 'file_prior_changes': 12,
#                 'method_prior_bugs': 0,
#                 'method_prior_changes': 0,
#                 'risky_lines': 3,
#                 'files': {
#                   'src/changes/changes.xml': {
#                     'id': 20,
#                     'prior_changes': 40,
#                     'prior_bugs': 0
#                   }, 
#                   'src/main/java/org/apache/commons/lang3/CharSetUtils.java': {
#                     'id': 21,
#                     'prior_changes': 12,
#                     'prior_bugs': 0
#                   },
#                   'src/test/java/org/apache/commons/lang3/CharSetUtilsTest.java': {
#                     'id': 22,
#                     'prior_changes': 5,
#                     'prior_bugs': 0
#                   }
#                 }
#               },
#               'b78295b4faca2fd26024a26ef4638b0dce5d40a3':
#               {
#                 'author_prior_changes': 246,
#                 'author_prior_bugs': 12,
#                 'risk_score': 0.0224,
#                 'bug_density': 0,
#                 'loc': 29,
#                 'la': 29,
#                 'bugs': 0,
#                 'diff': 29,
#                 'file_prior_bugs': 36,
#                 'file_prior_changes': 189,
#                 'method_prior_bugs': 0,
#                 'method_prior_changes': 0,
#                 'risky_lines': 0,
#                 'files': {
#                   'src/changes/changes.xml': {
#                     'id': 26,
#                     'prior_changes': 146,
#                     'prior_bugs': 0
#                   }, 
#                   'src/main/java/org/apache/commons/lang3/StringUtils.java': {
#                     'id': 27,
#                     'prior_changes': 189,
#                     'prior_bugs': 36
#                   },
#                   'src/test/java/org/apache/commons/lang3/StringUtilsTest.java': {
#                     'id': 28,
#                     'prior_changes': 72,
#                     'prior_bugs': 0
#                   }
#                 }
#               },
#               '16774d1c0d6d8aa4579f7c96b3fdb78bd118e5aa':
#               {
#                 'author_prior_changes': 541,
#                 'author_prior_bugs': 48,
#                 'risk_score': 0.0224,
#                 'bug_density': 0,
#                 'loc': 9,
#                 'la': 5,
#                 'bugs': 0,
#                 'diff': 5,
#                 'file_prior_bugs': 14,
#                 'file_prior_changes': 351,
#                 'method_prior_bugs': 0,
#                 'method_prior_changes': 0,
#                 'risky_lines': 0,
#                 'files': {
#                   'src/changes/changes.xml': {
#                     'id': 26,
#                     'prior_changes': 146,
#                     'prior_bugs': 0
#                   }, 
#                   'src/main/java/org/apache/commons/lang3/StringUtils.java': {
#                     'id': 27,
#                     'prior_changes': 189,
#                     'prior_bugs': 36
#                   },
#                   'src/test/java/org/apache/commons/lang3/StringUtilsTest.java': {
#                     'id': 28,
#                     'prior_changes': 72,
#                     'prior_bugs': 0
#                   }
#                 }
#               },
#               'b78295b4faca2fd26024a26ef4638b0dce5d40a3':
#               {
#                 'author_prior_changes': 246,
#                 'author_prior_bugs': 12,
#                 'risk_score': 0.0224,
#                 'bug_density': 0,
#                 'loc': 29,
#                 'la': 29,
#                 'bugs': 0,
#                 'diff': 29,
#                 'file_prior_bugs': 36,
#                 'file_prior_changes': 189,
#                 'method_prior_bugs': 0,
#                 'method_prior_changes': 0,
#                 'risky_lines': 0,
#                 'files': {
#                   'src/changes/changes.xml': {
#                     'id': 26,
#                     'prior_changes': 146,
#                     'prior_bugs': 0
#                   }, 
#                   'src/main/java/org/apache/commons/lang3/StringUtils.java': {
#                     'id': 27,
#                     'prior_changes': 189,
#                     'prior_bugs': 36
#                   },
#                   'src/test/java/org/apache/commons/lang3/StringUtilsTest.java': {
#                     'id': 28,
#                     'prior_changes': 72,
#                     'prior_bugs': 0
#                   }
#                 }
#               },
#               'b78295b4faca2fd26024a26ef4638b0dce5d40a3':
#               {
#                 'author_prior_changes': 246,
#                 'author_prior_bugs': 12,
#                 'risk_score': 0.0224,
#                 'bug_density': 0,
#                 'loc': 29,
#                 'la': 29,
#                 'bugs': 0,
#                 'diff': 29,
#                 'file_prior_bugs': 36,
#                 'file_prior_changes': 189,
#                 'method_prior_bugs': 0,
#                 'method_prior_changes': 0,
#                 'risky_lines': 0,
#                 'files': {
#                   'src/changes/changes.xml': {
#                     'id': 26,
#                     'prior_changes': 146,
#                     'prior_bugs': 0
#                   }, 
#                   'src/main/java/org/apache/commons/lang3/StringUtils.java': {
#                     'id': 27,
#                     'prior_changes': 189,
#                     'prior_bugs': 36
#                   },
#                   'src/test/java/org/apache/commons/lang3/StringUtilsTest.java': {
#                     'id': 28,
#                     'prior_changes': 72,
#                     'prior_bugs': 0
#                   }
#                 }
#               }
#               }


# users = {}
# changes = []
# files = []
# authors = []

# def get_change_info(id):
#   change = {}
#   url = "https://api.github.com/repos/apache/commons-lang/commits/{}".format(id)
#   try:
#     res = requests.get(url, headers=headers)
#     if res.status_code == 200:
#       data = res.json()
#     #   data = json.loads(res.text[6:-2])
#       change = {
#         "change_id": data["sha"],
#         "repo": "apache/commons-lang",
#         "branch": "master",
#         "subject": None,
#         "created": data["commit"]["author"]["date"],
#         "updated": data["commit"]["committer"]["date"],
#         "submitted": None,
#         "insertions": data["stats"]["additions"],
#         "deletions": data["stats"]["deletions"],
#         "number": None,
#         "author": data["commit"]["author"]["email"],
#         "commit_msg": data["commit"]["message"],
#         "parent": data["parents"][0]["sha"],
#         "project": "apache"
#       }
#       if data["commit"]["author"]["email"] not in users:
#           users[data["commit"]["author"]["email"]] = {
#             "name": data["commit"]["author"]["name"],
#             "email": data["commit"]["author"]["email"],
#           }
#     else:
#       print("Unexpected response code from REST server for change {}: {}".format(id, res.status_code), file=sys.stderr)
#   except Exception as e:
#     print("Unexpected exception for change {}: {}".format(id, e), file=sys.stderr)
#   return change

# def get_changed_files(id):
#   files = []
#   url = "https://api.github.com/repos/apache/commons-lang/commits/{}".format(id)
#   try:
#     res = requests.get(url, headers=headers)
#     if res.status_code == 200:
#       data = res.json()
#       parent = data["parents"][0]["sha"]
#       for file in data["files"]:
#         raw_url = file["raw_url"].replace(id, parent)
#         file_content = requests.get(raw_url, headers=headers).text
#         files.append({
#           "filename": file["filename"],
#           "status": None,
#           "insertions": file["additions"],
#           "deletions": file["deletions"],
#           "code_a": file_content,
#           "code_b": None,
#           "diff": "--- a/{}\n +++ b/{}\n".format(file["filename"], file["filename"]) + file["patch"]
#         })
#     else:
#       print("Unexpected response code from REST server for change {}: {}".format(id, res.status_code), file=sys.stderr)
#   except Exception as e:
#     print("Unexpected exception for change {}: {}".format(id, e), file=sys.stderr)
#   return files

# file_diff = {}
# for id, info in data.items():
#   change = get_change_info(id)
#   change["author_prior_changes"] = info["author_prior_changes"]
#   change["author_prior_bugs"] = info["author_prior_bugs"]
#   change["risk_score"] = info["risk_score"]
#   change["bug_density"] = info["bug_density"]
#   changes.append(change)
#   change_files = get_changed_files(id)
#   # file_diff = {}
#   for file in change_files:
#     file["change_id"] = id
#     if file["filename"] in info["files"]:
#       file["id"] = info["files"][file["filename"]]["id"]
#       file["prior_bugs"] = info["files"][file["filename"]]["prior_bugs"]
#       file["prior_changes"] = info["files"][file["filename"]]["prior_changes"]
#       files.append(file)


# author_id = 1
# users_new = {}
# for email, info in users.items():
#   users_new[email] = {
#     "account_id": author_id,
#     "name": info["name"],
#     "email": info["email"],
#     "project": "apache"
#   }
#   author_id += 1
# users = users_new
# authors = list(users.values())

# print(changes)
# # print(files)
# print(users)
# print(len(changes))
# print(len(files))
# print(len(authors))
# # print(file_diff['src/corelib/thread/qsemaphore.cpp']['code_a'])

